# Кратко описание на задачата
> Система за препоръки на книги, като препоръките се базират на рейтинг от читателите/потребителите. На данните би могло да се приложи ре-филтрация по възраст на читателите. Основен проблем, който би следвало да се реши преди ре-филтрацията е, че около 40% от читателите не са посочили възрастта си, което води до нужда от поправки/зачистване в данните преди ре-филтрацията, което на по-късен етап установихме, че води до загуба на информация и неточности в модела. Естествено, в следната документация предлагаме пълния процес, през който преминахме в изграждането на система за препоръки на книги.
>
> Забележка: Преди да дсотигнем крайната реализация на задачата си преминахме през няколко различни потенциални подхода за решаване на подобна задача, но основен подход, на който се спряхме, за реализация на задачата ни е k-NN алгоритъма във варианта му на a-NN (k-NN алгоритъм с приближение). За да изведем съображенията, които ни насочиха да тръгнем в тази посока би било редно да опишем подходите и експериментите, които проведохме, заеднос техните резултати и изводи, както и да покажем с какво a-NN подхода превъзхожда всички останали, изпробвани алгоритми в условностите на задачата ни. Повече информация за процеса на имплементация, както и за преценката за нужния ни алгоритъм може да намерите в графа Използвани алгоритми на документацията.
>
> **Забележка**: Кодът е разработен на Python със съответно необходимите му библиотеки като:
>
> 1. Numpy
> 2. pandas
> 3. sklearn.neighbors
> 4. scikit-learn (от sklearn.neighbors)
> 5. sklearn.metrics
> 6. sklearn.model_selection
> 7. scipy.stats
> 8. scipy.sparse
> 9. warnings

# Използвани данни
> Данните, които разглеждаме като CSV файлове са предоставени от **Amazon Web Services**.

## Users
Информацията, която се съхранява за читателите е техните уникални идентификационни номера (User-ID), локациите им (Location) и възрастта им.

## Books
Информацията, която се съхранява за книгите включва ISBN номерата им, загавие (Book-Title), автор (Book-Author), година на публикаци (Year-Of-Publication) и издателство на книгата (Publisher).

## Ratings
Информацията, която се съхранява за оценките на книгите включва кой потребител (User-ID) коя книга (ISBN) а е оценил и каква е била самата оценка (число в интервала от 1 до 10, вписано в полето Book-Rating).

# Организация на файловете
- docs - папка, която съдържа документацията на проекта (, тоест текущо разглеждания от Вас файл), както и папка resources и прилежащите й файлове, необходии за изграждането на документацията на проекта.
    - resources - папка, съдържаща всички необходими ресурсни файлове за създаването на документацията на проекта (например скрийншотите, включени в нея).
- public- папка, съдържаща глобално достъпни (и от двамата разработчици) файлове, необходими за осмислянето на задачата и построението на решението.
- src - папка, съдържаща данните за обработка, изходния код на системата и Jupyter notebook-а, описващ достатъчно подробно кода и прилежащите му примери. Цялото това съдържание условно се разделя в следните три папки:
    - code - папка, която съдържа изходния код на всички проведени експерименти, както и изходния код на крайния алгоритъм, използван за решаването на проблема.
    - data - папка, съдържаща всички CSV таблици, използвани в процеса на разработка на системата.
    - jupyter - папка, съдържаща ipynb файла на въпросния Jupyter notebook, използван за демонстрация на работата на системата и документация на самата демонстрация.

# Използвани алгоритми
## Експерименти

### Експеримент k-NN алгоритъм 

Импортиране на използваните dataset-ове.


In [1]:
import pandas as pd
import numpy as np
from pprint import pprint as pp
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

df_ratings = pd.read_csv("../data/Ratings.csv", na_values=["null", "nan", ""])
df_books = pd.read_csv(
    "../data/Books.csv",
    na_values=["null", "nan", ""],
    usecols=["ISBN", "Book-Title", "Book-Author"],
)
df_users = pd.read_csv("../data/Users.csv", na_values=["null", "nan", ""])

df_books = df_books.fillna("NaN")
df_ratings = df_ratings.dropna()

#### Преброяване на броя рейтинги, които са получили книгите:

- групиране на рейтингите по ISBN;
- преброяване на рейтингите;


In [2]:
combine_book_ratings = pd.merge(df_ratings, df_books, on="ISBN")
combine_book_ratings = combine_book_ratings.drop(["Book-Author"], axis="columns")

book_rating_count = (
    combine_book_ratings.groupby(by=["ISBN"])["Book-Rating"]
    .count()
    .reset_index()
    .rename(columns={"Book-Rating": "RatingCount"})
)[["ISBN", "RatingCount"]]

#### Извличане на най-оценяваните книги:

- Извличаме стойностите на квантилите на броя рейтинги в 90% на 100%.
- За статистическа значимост избираме топ 10% най-оценявани книги.


In [3]:
book_rating_with_total_count = combine_book_ratings.merge(
    book_rating_count, on=["ISBN"], how="left"
)
pp(book_rating_with_total_count["RatingCount"].quantile(np.arange(0.9, 1, 0.01)))
# Top 10% of rating counts
popularity_threshold = 136

rating_popular_books = book_rating_with_total_count.query(
    "RatingCount >= @popularity_threshold"
)

0.90    136.0
0.91    150.0
0.92    167.0
0.93    184.0
0.94    209.0
0.95    236.0
0.96    277.0
0.97    350.0
0.98    420.0
0.99    568.0
Name: RatingCount, dtype: float64


#### Дефиниране на k-NN модела:

- Използваме пивот таблиза за дефиниране на модела
  - Индекс: ISBN
  - Колони: User-ID
  - Стойности: Book-Rating
- k-NN модел
  - метрика: cosine
  - алгоритъм: auto


In [4]:
pivot = (
    rating_popular_books.drop_duplicates(["Book-Title", "User-ID"])
    .pivot(index="ISBN", columns="User-ID", values="Book-Rating")
    .fillna(0)
)

model_knn = NearestNeighbors(metric="cosine", algorithm="auto")
model_knn.fit(csr_matrix(pivot.values))

NearestNeighbors(metric='cosine')

#### Функция за прпоръки на книги:

- Използваме ISBN като индексация на пивот матрицата (таблицата).
- Използваме модела k-NN от `sklearn.neighbors`.
- След това групираме ISBN-ите на съседите и разстоянията от тях до търсената книга. 
- При грешка връщаме съобщение, че дадената книга не е сред топ 10% на оценяваните книги.


In [5]:
def get_recommends(isbn="", k_neighbors=5):
    try:
        x = pivot.loc[isbn].array.reshape(1, -1)
        distances, indices = model_knn.kneighbors(x, n_neighbors=k_neighbors)
        R_books = []
        for distance, indice in zip(distances[0], indices[0]):
            if distance != 0:
                R_book = combine_book_ratings[
                    combine_book_ratings["ISBN"] == pivot.index[indice]
                ]["Book-Title"].values[0]
                R_books.append([R_book, distance])
        recommended_books = [isbn, R_books[::-1]]
        return recommended_books
    except:
        return f"{isbn} is not in the top books"

#### Тест на системата за препоръки на книги

    Използвайки популярни ISBN-и на дадени книги, ние можем да открием най-сходните с тях.


In [6]:
pp(get_recommends("1558745157"))

['1558745157',
 [["Left Behind: A Novel of the Earth's Last Days (Left Behind No. 1)",
   np.float64(0.9381235623430844)],
  ['Night Sins', np.float64(0.9352435834429496)],
  ['On the Street Where You Live', np.float64(0.9258003569774721)],
  ['A Child Called \\It\\": One Child\'s Courage to Survive"',
   np.float64(0.7356135842239919)],
  ["The Lost Boy: A Foster Child's Search for the Love of a Family",
   np.float64(3.3306690738754696e-16)]]]


### Експеримент k-NN алгоритъм с регресорна препоръка на книги и грануларно изчисление на дистанциите (между текущо изследвания пример и неговите k на брой най-близки съседи)

#### Обединяваме данните, които ще са ни неоходими:
- всички данни от таблицата с рейтингите и годините на читателите.
    - Годините на читателите са ни е неоходими, тъй като искаме да направим **по-комплексна препоръка** на база **рейтингите на книгите и възрастите на читателите**.
- проверяваме колко от читателите не са си споделили възрастта 
    - според статистика на сайта, от който взехме данните, такива читатели са около 40% от всички.

In [ ]:
from scipy.stats import pearsonr

merged_data = pd.merge(
    df_ratings.dropna(), df_users[["User-ID", "Age"]], on="User-ID", how="inner"
)

print(f"Merged data:\n{merged_data}\n")
print(f"Merged data info:\n{merged_data.info()}\n")

#### Запълваме NaN възрастите на потребителите с невалидни такива (тоест -1).
- Проверяваме дали всички колони в **merged_data** са с еднаква дължина, защото това ще ни е неоходима стъпка за регресията

In [ ]:
merged_data = merged_data.fillna(value=-1)
print(f"Cleaned of NaN values merged data info:\n{merged_data.info()}\n")

#### Подготовка на векторите, необходими за регресионния модел.
- Правим корелация по метода на Пиърсън, за да проврим съществува ли линейна връзка между рейтингите на книгите и възрастовите групи, очертали се сред нашите читатели.
    - Установяваме, че липсва линейна връзка между тези два показателя , тъй като p-value-то е равно на 0.0, тоест регресионния модел не би бил удачен избор за продължение.
- В кода си сме доразвили този процес като чист експеримент, който доведе до много лоши препоръки в потвържение на резултатите, до които доведе и корелацията. За повече детайли за този експеримент, може да разгледате кода в knn_with_regression_and_granular_computation.py в директорията src/code, както и документацията на проекта за разяснение на кода.

In [ ]:
x_ratings = np.array(merged_data["Book-Rating"])
y_ages = np.array(merged_data["Age"])

correlation, p_value = pearsonr(x_ratings, y_ages)

print(f"Pearson Correlation: {correlation}")
print(
    f"P-value: {p_value}"
)  

### Експеримент a-NN
#### a-NN: HNSW разновидност

#### a-NN: FAISS разновидност

#### a-NN: ANNOY разновидност

#### Сравнение между трите вида a-NN имплементации

# Конфигурация на проекта

- **Windows / MacOs / Linux**
    - **Стъпка 1**: Направете директория, в която искате да разположите кода ни.
    - **Стъпка 2**: Клонирайте следното repository: https://github.com/Nv4n/soz-project-2024-2025.git на вашата машина в току-що създадената от Вас директория. Клонирането може да стане през:
        - терминала със следната команда: git clone със задаване на директория. Когато клонирате хранилище, можете директно да зададете целева директория, където да бъде изтеглено: git clone <URL-на-репото> <път-към-директорията>
        - UI-я на някое IDE (например Visual Studio Code).
    - **Стъпка 3**: Инсталирайте някой Python Extension Pack и нео=бходимите библиотеки, описани по-нагоре в този документ.
    - **Стъпка 4**: Отворете някой от файловете с изходния код, стартирайте го и експериментирайте.

> **Забележка**: И за трите операционни системи общите стъпки за конфигурация са еднакви. Разликите могат да идват от това какъв терминал използва самата операционна система или как се извикват конкретни команди.